In [ ]:
%load_ext autoreload
%autoreload 2
import torch
from models.vgg19style import VGG19Style, VGG19Style2
from lib.fashionpedia_processed import FashionPediaProcessed

In [ ]:
# load model
model = VGG19Style2(32)

In [ ]:
# load dataset
data = FashionPediaProcessed()
data_loader = torch.utils.data.DataLoader(
    data, batch_size=16, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(data[2]['img'].permute(1, 2, 0))
plt.show()

In [ ]:
from tqdm.autonotebook import tqdm

optimizer = torch.optim.Adam(model.parameters(), lr= 0.0001)

def train_one_epoch(epoch_index):
    running_loss = 0.
    num_batches = 0
    last_loss = -1

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, dat in enumerate(tqdm(data_loader)):
        # Every data instance is an input + label pair

        print(dat) 
        inputs = dat['img']

        print(inputs.shape)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = torch.nn.BCELoss(outputs, dat['att_oh'])
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        num_batches += 1

        if i % 100 == 99:
            last_loss = running_loss / 100  # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(data_loader) + i + 1
            print('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss


# Initializing in a separate cell so we can easily add more epochs to the same run
EPOCHS = 200

best_test_loss = 1_000_000.

for epoch in range(EPOCHS):
    print('\nEPOCH {}:'.format(epoch + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch)

    print(f'LOSS train {avg_loss}')

    # Log the running loss averaged per batch
    # for both training and validation
    print('Training vs. Validation Loss',
            {'Training': avg_loss},
            epoch + 1)

    epoch += 1

In [ ]:
def content_loss(target, generated):
	layers = [] # Todo layers
	total = 0
	for l in layers:
		loss = nn.MSELoss()
		total += loss(target[l], generated[l])
	
	return total*0.5

In [ ]:
def style_loss(style, generated):
	layers = []  # Todo layers
	total = 0
	weight = 1 / len(layers) # Todo weight
	for l in layers:
		loss = nn.MSELoss()
		output = loss(style[l], generated[l])
		N = l.N # Todo number of distinct filter/feature maps in layer
		M = l.M # Todo height times width of feature maps
		total += weight * output / (4 * N**2 * M**2)

	return total

In [ ]:
def total_loss(target, style, generated, alpha, beta):
    return alpha * content_loss(target, generated) + beta * style_loss(style, generated)